In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Acute_Myeloid_Leukemia"
cohort = "GSE235070"

# Input paths
in_trait_dir = "../../input/GEO/Acute_Myeloid_Leukemia"
in_cohort_dir = "../../input/GEO/Acute_Myeloid_Leukemia/GSE235070"

# Output paths
out_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/GSE235070.csv"
out_gene_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE235070.csv"
out_clinical_data_file = "../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE235070.csv"
json_path = "../../output/preprocess/Acute_Myeloid_Leukemia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Ferritinophagy is a Druggable Vulnerability of Quiescent Leukemic Stem Cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['disease state: patient with AML']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
import numpy as np
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on the background information, this appears to be primarily about leukemic stem cells.
# However, without more specific information about the data structure, we should be cautious.
is_gene_available = True  # Assuming gene expression data is available for AML studies

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
trait_row = 0  # The dataset has AML disease state information (row 0)
age_row = None  # Age information is not provided in the sample characteristics
gender_row = None  # Gender information is not provided in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert AML trait value to binary (1 = has AML, 0 = control)"""
    if value is None:
        return None
    
    # Extract the value part (after colon if present)
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check if the value indicates AML
    if 'AML' in value.upper() or 'ACUTE MYELOID LEUKEMIA' in value.upper() or 'patient' in value.lower():
        return 1
    elif 'control' in value.lower() or 'healthy' in value.lower() or 'normal' in value.lower():
        return 0
    else:
        return None

def convert_age(value):
    """Convert age to numerical value"""
    # This function is included for completeness, but age data is not available
    if value is None:
        return None
    
    # Extract the value part (after colon if present)
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract numerical age
    try:
        # Extract digits if embedded in text
        import re
        age_match = re.search(r'(\d+)', value)
        if age_match:
            return float(age_match.group(1))
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 = female, 1 = male)"""
    # This function is included for completeness, but gender data is not available
    if value is None:
        return None
    
    # Extract the value part (after colon if present)
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    
    if isinstance(value, str):
        value = value.lower()
        if 'female' in value or 'f' == value:
            return 0
        elif 'male' in value or 'm' == value:
            return 1
    
    return None

# 3. Save Metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Assuming clinical_data is available from previous steps
    try:
        # Check if clinical_data is available
        if 'clinical_data' in locals() or 'clinical_data' in globals():
            # Extract clinical features
            selected_clinical_df = geo_select_clinical_features(
                clinical_df=clinical_data,
                trait=trait,
                trait_row=trait_row,
                convert_trait=convert_trait,
                age_row=age_row,
                convert_age=convert_age if age_row is not None else None,
                gender_row=gender_row,
                convert_gender=convert_gender if gender_row is not None else None
            )
            
            # Preview the extracted clinical features
            preview = preview_df(selected_clinical_df)
            print("Preview of selected clinical features:")
            print(preview)
            
            # Create directory if it doesn't exist
            os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
            
            # Save the extracted clinical features
            selected_clinical_df.to_csv(out_clinical_data_file, index=False)
            print(f"Clinical data saved to {out_clinical_data_file}")
        else:
            print("Clinical data not available from previous steps")
    except NameError:
        print("Clinical data not available from previous steps")
else:
    print("Trait data not available, skipping clinical feature extraction")


Preview of selected clinical features:
{'GSM7494217': [1.0], 'GSM7494218': [1.0], 'GSM7494219': [1.0], 'GSM7494220': [1.0], 'GSM7494221': [1.0], 'GSM7494222': [1.0], 'GSM7494223': [1.0], 'GSM7494224': [1.0], 'GSM7494249': [1.0], 'GSM7494250': [1.0], 'GSM7494251': [1.0], 'GSM7494252': [1.0], 'GSM7494253': [1.0], 'GSM7494254': [1.0], 'GSM7494255': [1.0], 'GSM7494256': [1.0], 'GSM7494509': [1.0], 'GSM7494510': [1.0], 'GSM7494511': [1.0], 'GSM7494512': [1.0], 'GSM7494513': [1.0], 'GSM7494514': [1.0], 'GSM7494515': [1.0], 'GSM7494516': [1.0], 'GSM7494517': [1.0], 'GSM7494518': [1.0], 'GSM7494519': [1.0], 'GSM7494520': [1.0], 'GSM7494521': [1.0], 'GSM7494522': [1.0], 'GSM7494523': [1.0], 'GSM7494524': [1.0]}
Clinical data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE235070.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1',
       'TC0100006480.hg.1', 'TC0100006483.hg.1', 'TC0100006486.hg.1',
       'TC0100006490.hg.1', 'TC0100006492.hg.1', 'TC0100006494.hg.1',
       'TC0100006497.hg.1', 'TC0100006499.hg.1', 'TC0100006501.hg.1',
       'TC0100006502.hg.1', 'TC0100006514.hg.1', 'TC0100006516.hg.1',
       'TC0100006517.hg.1', 'TC0100006524.hg.1', 'TC0100006540.hg.1',
       'TC0100006548.hg.1', 'TC0100006550.hg.1'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers "TC0100006437.hg.1" appear to be Affymetrix transcript cluster IDs 
# from a microarray platform, not standard human gene symbols.
# They need to be mapped to human gene symbols for better interpretability and consistency.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 // ENSEMBL // olfactory receptor, family 4, subfamily F, member 5 [gene_biotype:protein_coding transcript_biotype:protein_coding] // chr1 // 10

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Analyze the gene annotation dataframe to identify columns containing probe IDs and gene symbols
# From the preview, we can see that 'ID' matches the gene identifiers in gene_data's index (TC01... format)
# For gene symbols, we need to extract them from the 'SPOT_ID.1' column which contains transcript info

# 2. Create the gene mapping dataframe using the appropriate columns
# ID column already matches the probe IDs
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'SPOT_ID.1')

# 3. Apply the gene mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the mapped data
print(f"Original data shape (probes): {len(gene_data.index)} genes × {len(gene_data.columns)} samples")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])


Original data shape (probes): 85633 genes × 32 samples
First 5 gene symbols after mapping:
Index(['A-', 'A-1', 'A-2', 'A-52', 'A-E'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Let's first check what's actually in the clinical_data to avoid errors
print("Clinical data shape:", clinical_data.shape)
print("Sample characteristics dictionary:")
sample_characteristics_dict = get_unique_values_by_row(clinical_data)
print(sample_characteristics_dict)

# Define the trait conversion function based on the actual available data
def convert_trait(value):
    """Convert AML status to binary (1 = has AML, 0 = control/healthy)"""
    if value is None:
        return None
    
    # Extract the value part (after colon if present)
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # In this dataset, all samples appear to be AML patients
    if 'AML' in value.upper() or 'patient' in value.lower():
        return 1
    elif 'control' in value.lower() or 'healthy' in value.lower() or 'normal' in value.lower():
        return 0
    else:
        return None

# Use the correct row index based on the sample characteristics dictionary
trait_row = 0  # The only row available (disease state: patient with AML)
age_row = None  # Age information not available
gender_row = None  # Gender information not available

# Check if clinical_data actually contains data before proceeding
if clinical_data.shape[0] > 0:
    # Extract clinical features
    selected_clinical_data = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,
        convert_age=None,
        gender_row=None,
        convert_gender=None
    )
    
    print("Clinical data preview:")
    print(preview_df(selected_clinical_data))
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_data.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # 2. Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_data, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")

    # 3. Handle missing values in the linked data
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Data shape after handling missing values: {linked_data.shape}")

    # 4. Determine whether the trait and some demographic features are severely biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

    # 5. Conduct quality check and save the cohort information
    note = "Dataset contains only AML patients without controls, which may limit its utility for some analyses"
    is_usable = validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=True, 
        is_biased=is_trait_biased, 
        df=unbiased_linked_data,
        note=note
    )

    # 6. If the linked data is usable, save it
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Processed dataset saved to {out_data_file}")
    else:
        print("Dataset not usable due to bias in trait distribution. Data not saved.")
else:
    print("No clinical data available. Cannot proceed with linking and subsequent steps.")
    # Still need to save the cohort info indicating the dataset isn't usable
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False, 
        is_biased=None, 
        df=pd.DataFrame(),
        note="Dataset doesn't contain usable clinical data for trait analysis"
    )

Normalized gene data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/gene_data/GSE235070.csv
Clinical data shape: (1, 33)
Sample characteristics dictionary:
{0: ['disease state: patient with AML']}
Clinical data preview:
{'GSM7494217': [1.0], 'GSM7494218': [1.0], 'GSM7494219': [1.0], 'GSM7494220': [1.0], 'GSM7494221': [1.0], 'GSM7494222': [1.0], 'GSM7494223': [1.0], 'GSM7494224': [1.0], 'GSM7494249': [1.0], 'GSM7494250': [1.0], 'GSM7494251': [1.0], 'GSM7494252': [1.0], 'GSM7494253': [1.0], 'GSM7494254': [1.0], 'GSM7494255': [1.0], 'GSM7494256': [1.0], 'GSM7494509': [1.0], 'GSM7494510': [1.0], 'GSM7494511': [1.0], 'GSM7494512': [1.0], 'GSM7494513': [1.0], 'GSM7494514': [1.0], 'GSM7494515': [1.0], 'GSM7494516': [1.0], 'GSM7494517': [1.0], 'GSM7494518': [1.0], 'GSM7494519': [1.0], 'GSM7494520': [1.0], 'GSM7494521': [1.0], 'GSM7494522': [1.0], 'GSM7494523': [1.0], 'GSM7494524': [1.0]}
Clinical data saved to ../../output/preprocess/Acute_Myeloid_Leukemia/clinical_data/GSE235070.csv


Data shape after handling missing values: (32, 19976)
Quartiles for 'Acute_Myeloid_Leukemia':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Acute_Myeloid_Leukemia' in this dataset is severely biased.

Dataset not usable due to bias in trait distribution. Data not saved.
